In [ ]:
!pip install matplotlib

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
from tensorflow.keras import mixed_precision


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Load and preprocess the CSV data
csv_path = '/Brain_MRI_Dataset/mri_dataset/Brain_MRI_conditions.csv'
csv_data = pd.read_csv(csv_path)

# Basic preprocessing of the CSV dataset
# Keep 'Type', 'Description', and 'Body Part' columns for now
csv_data = csv_data[['Type', 'Description', 'Body Part']]
csv_data.fillna('', inplace=True)

# Encode categorical columns
label_encoders = {}
for col in csv_data.columns:
    le = LabelEncoder()
    csv_data[col] = le.fit_transform(csv_data[col])
    label_encoders[col] = le

# Paths to the tumor and no tumor image folders
tumor_folders = ['/Brain_MRI_Dataset/Training/glioma', '/Brain_MRI_Dataset/Training/meningioma', '/Brain_MRI_Dataset/Training/pituitary']
no_tumor_folder = '/Brain_MRI_Dataset/Training/notumor'

# Function to load and preprocess images from a folder
def load_images_from_folder(folder, label, img_size=(128, 128)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, img_size)
            img = img / 255.0  # Normalize to 0-1 range
            images.append(img)
            labels.append(label)
    return images, labels

# Load tumor and no tumor images from multiple folders
X_images = []
y_labels = []

# Load tumor images from the 3 tumor folders
for tumor_folder in tumor_folders:
    tumor_images, tumor_labels = load_images_from_folder(tumor_folder, label=1)  # Label 1 for tumor
    X_images.extend(tumor_images)
    y_labels.extend(tumor_labels)

# Load no tumor images
no_tumor_images, no_tumor_labels = load_images_from_folder(no_tumor_folder, label=0)  # Label 0 for no tumor
X_images.extend(no_tumor_images)
y_labels.extend(no_tumor_labels)

# Convert lists to numpy arrays
X_images = np.array(X_images)
y_labels = np.array(y_labels)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_images, y_labels, test_size=0.2, random_state=42)

# Build a CNN model for image classification
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification: cancer or no cancer
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Save the model
model.save('brain_mri_cancer_model.h5')

# To make predictions
def predict_cancer(img_path, model):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (128, 128))
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    prediction = model.predict(img)
    if prediction > 0.5:
        return "Cancer"
    else:
        return "No Cancer"

# Example prediction
new_image_path = '/Brain_MRI_Dataset/Testing/glioma/Te-glTr_0000.jpg'
result = predict_cancer(new_image_path, model)
print(result)


In [ ]:
new_image_path = 'Brain_MRI_Dataset/Testing/notumor/Te-noTr_0000.jpg'
result = predict_cancer(new_image_path, model)
print(result)

In [ ]:
new_image_path = '/Brain_MRI_Dataset/Testing/glioma/Te-gl_0294.jpg'
result = predict_cancer(new_image_path, model)
print(result)

In [ ]:
import matplotlib.pyplot as plt

# Function to plot accuracy and loss curves
def plot_accuracy_loss(history):
    # Plotting the training and validation accuracy
    plt.figure(figsize=(12, 5))

    # Accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Show plots
    plt.tight_layout()
    plt.show()

# Train the model and save history
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Plot the accuracy and loss curves
plot_accuracy_loss(history)


## **Use Parallel Process**

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Load and preprocess the CSV data
csv_path = '/Brain_MRI_Dataset/Brain_MRI_conditions.csv'
csv_data = pd.read_csv(csv_path)

# Basic preprocessing of the CSV dataset
# Keep 'Type', 'Description', and 'Body Part' columns for now
csv_data = csv_data[['Type', 'Description', 'Body Part']]
csv_data.fillna('', inplace=True)

# Encode categorical columns
label_encoders = {}
for col in csv_data.columns:
    le = LabelEncoder()
    csv_data[col] = le.fit_transform(csv_data[col])
    label_encoders[col] = le

# Paths to the tumor and no tumor image folders
tumor_folders = ['/Brain_MRI_Dataset/Training/glioma',
                 '/Brain_MRI_Dataset/Training/meningioma',
                 '/Brain_MRI_Dataset/Training/pituitary']
no_tumor_folder = '/Brain_MRI_Dataset/Training/notumor'

# Function to load and preprocess a single image (helper function)
def process_single_image(file_info):
    folder, filename, label, img_size = file_info
    img_path = os.path.join(folder, filename)
    img = cv2.imread(img_path)
    if img is not None:
        img = cv2.resize(img, img_size)
        img = img / 255.0  # Normalize to 0-1 range
        return img, label
    return None, None

# Function to load and preprocess images from multiple folders in parallel
def load_images_parallel(folders, labels, img_size=(128, 128)):
    images = []
    labels_list = []
    all_files = []

    # Prepare the list of files to process
    for folder, label in zip(folders, labels):
        for filename in os.listdir(folder):
            all_files.append((folder, filename, label, img_size))

    # Use ThreadPoolExecutor to load images in parallel
    with ThreadPoolExecutor() as executor:
        results = executor.map(process_single_image, all_files)

    # Collect valid images and labels from the results
    for img, label in results:
        if img is not None:
            images.append(img)
            labels_list.append(label)

    return np.array(images), np.array(labels_list)

# Load tumor and no tumor images in parallel
tumor_folders_labels = [1] * len(tumor_folders)  # Tumor: label 1
folders = tumor_folders + [no_tumor_folder]
labels = tumor_folders_labels + [0]  # No tumor: label 0

# Load images from tumor and no tumor folders
X_images, y_labels = load_images_parallel(folders, labels)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_images, y_labels, test_size=0.2, random_state=42)

# Build a CNN model for image classification
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification: cancer or no cancer
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Save the model
model.save('brain_mri_cancer_model.h5')

# To make predictions
def predict_cancer(img_path, model):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (128, 128))
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    prediction = model.predict(img)
    if prediction > 0.5:
        return "Cancer"
    else:
        return "No Cancer"

# Example prediction
new_image_path = '/Brain_MRI_Dataset/Testing/glioma/Te-glTr_0000.jpg'
result = predict_cancer(new_image_path, model)
print(result)


1.35s

## **Performance Check**

In [ ]:
import time

# Serial version
def load_images_serial(folders, labels, img_size=(128, 128)):
    images = []
    labels_list = []

    for folder, label in zip(folders, labels):
        for filename in os.listdir(folder):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, img_size)
                img = img / 255.0  # Normalize to 0-1 range
                images.append(img)
                labels_list.append(label)

    return np.array(images), np.array(labels_list)

# Measure time for serial loading
start_time_serial = time.time()
X_images_serial, y_labels_serial = load_images_serial(folders, labels)
end_time_serial = time.time()
serial_time = end_time_serial - start_time_serial
print(f"Serial execution time: {serial_time:.2f} seconds")


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Load and preprocess the CSV data
csv_path = '/content/drive/MyDrive/Project_Dataset/Brain_MRI_conditions.csv'
csv_data = pd.read_csv(csv_path)

# Basic preprocessing of the CSV dataset
csv_data = csv_data[['Type', 'Description', 'Body Part']]
csv_data.fillna('', inplace=True)

# Encode categorical columns
label_encoders = {}
for col in csv_data.columns:
    le = LabelEncoder()
    csv_data[col] = le.fit_transform(csv_data[col])
    label_encoders[col] = le

# Paths to the tumor and no tumor image folders
tumor_folders = ['/Brain_MRI_Dataset/Training/glioma',
                 '/Brain_MRI_Dataset/Training/meningioma',
                 '/Brain_MRI_Dataset/Training/pituitary']
no_tumor_folder = '/Brain_MRI_Dataset/Training/notumor'

# Function to load and preprocess a single image
def process_single_image(file_info):
    folder, filename, label, img_size = file_info
    img_path = os.path.join(folder, filename)
    img = cv2.imread(img_path)
    if img is not None:
        img = cv2.resize(img, img_size)
        img = img / 255.0  # Normalize to 0-1 range
        return img, label
    return None, None

# Function to load images in parallel
def load_images_parallel(folders, labels, img_size=(128, 128)):
    images = []
    labels_list = []
    all_files = []

    for folder, label in zip(folders, labels):
        for filename in os.listdir(folder):
            all_files.append((folder, filename, label, img_size))

    # Use ThreadPoolExecutor to load images in parallel
    with ThreadPoolExecutor() as executor:
        results = executor.map(process_single_image, all_files)

    for img, label in results:
        if img is not None:
            images.append(img)
            labels_list.append(label)

    return np.array(images), np.array(labels_list)

# Measure time for parallel loading
folders = tumor_folders + [no_tumor_folder]
tumor_folders_labels = [1] * len(tumor_folders)  # Tumor: label 1
labels = tumor_folders_labels + [0]  # No tumor: label 0

start_time_parallel = time.time()
X_images_parallel, y_labels_parallel = load_images_parallel(folders, labels)
end_time_parallel = time.time()
parallel_time = end_time_parallel - start_time_parallel
print(f"Parallel execution time: {parallel_time:.2f} seconds")

# Measure time for serial loading
def load_images_serial(folders, labels, img_size=(128, 128)):
    images = []
    labels_list = []

    for folder, label in zip(folders, labels):
        for filename in os.listdir(folder):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, img_size)
                img = img / 255.0  # Normalize to 0-1 range
                images.append(img)
                labels_list.append(label)

    return np.array(images), np.array(labels_list)

# Measure time for serial loading
start_time_serial = time.time()
X_images_serial, y_labels_serial = load_images_serial(folders, labels)
end_time_serial = time.time()
serial_time = end_time_serial - start_time_serial
print(f"Serial execution time: {serial_time:.2f} seconds")

# Plot the performance comparison
times = [serial_time, parallel_time]
methods = ['Serial', 'Parallel']

plt.figure(figsize=(8, 5))
plt.bar(methods, times, color=['blue', 'green'])
plt.xlabel('Method')
plt.ylabel('Execution Time (seconds)')
plt.title('Performance Comparison: Serial vs Parallel Image Loading')
plt.show()


In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


In [ ]:
# Enable mixed precision if supported
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
